I processed the data I scraped from ifcb so we can do the classification using image features.
- Now trying to figure how to use ifcb and enviromental features I collected.
-   The issue is that env variables are binned by day. 

So I kind of messed up collecting the data as I didn't get some important metadata like:
```
Triggers: 919
Images: 930
Triggers / s: 0.765
Volume Analyzed: 4.687 ml
ROIs / ml: 198.432
Size: 6.86 MB
Latitude: 32.86671
Longitude: -117.25587
```
Which is seemingly only available in that little side bar and not in a file (?)

- From my minimal research and knowledge it seems like you're supposed to do cells/mL as absolute abundance but I didn't get that data so..
- supposedly you can use the amount of L.poly / ROI in a sample as relative abundance since it's a ratio
    - something somethign fraction of ROI's that belong to a taxon.. 

So essentially what I am doing in this notebook is for each ifcb bin getting:
```
timestamp
Lpoly_fraction
roiCount
```

Then bin by day which will give the daily mean fraction of L. Polyedra
___

One thing that I am confused about is whether to use thresholds to determine if the taxa is present or not.
- like say 'A >.8'

my AI chatbot of choice suggested using the sum of probabilies 
- This is because autoclass calcuates the percent confidence (prb) of it being the taxa
- so supposedly when summing probabilties, you get the expected value contribution of that particle
    - which is linearity of expectation 

I need to review statistics lmao. I'm so bad at this.

___

So with the data I have this is what needs to be done??
- Image features(from ifcb) -> classification finding important image features..
- Daily relative abundances -> compare with enviromental features 

In [ ]:
from datetime import datetime
from pathlib import Path
import pandas as pd
import glob
import os
s = datetime.now()

base_path = "./ifcb_downloads/"
burger = Path(base_path) # used for iterating months I'm kinda hungry
out_dir = Path("./processed2/")
out_dir.mkdir(parents=True, exist_ok=True)

def read_hdr(path: Path) -> dict:
    return { k.strip(): v.strip()
            for line in path.read_text().splitlines()
            if ":" in line
            for k, v in [line.split(":", 1)] }

# iterate through each month directory
for month_dir in sorted(burger.iterdir()):
    path = base_path + month_dir.name + "/"
    print(f"Processing files in {path}... {(datetime.now()- s).total_seconds()}s")
    
    # for each hdr file in month folder get date and find matching files (with date)
    for files in glob.glob(path + '*.hdr'):
        date = os.path.basename(files).split('_')[0].lstrip('D')
        # read class scores file and extract relevant columns
        for match in glob.glob(os.path.join(path, f"*{date}*")):
            if match.endswith('.csv'):
                if 'class_scores' in match:
                    try:
                        class_scores = pd.read_csv(Path(match), usecols=['pid', 'Lingulodinium_polyedra'])
                        lpoly_fraction = (class_scores["Lingulodinium_polyedra"].sum() / len(class_scores))
                    except pd.errors.EmptyDataError:
                        class_scores = pd.DataFrame()
                        print(f"Class scores file is empty for date {date}")
            elif match.endswith('hdr'):
                hdr = read_hdr(Path(match))
                roi_count = float(hdr["roiCount"])

print(f"Processing complete in {(datetime.now()- s).total_seconds()}s yay")